In [1]:
import os
import cv2
import threading
import concurrent.futures
from PIL import Image
from tqdm import tqdm

def video_to_gif(video_path, output_folder):
    # Đọc video bằng OpenCV
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame về 512x512
        frame = cv2.resize(frame, (512, 512))
        # Chuyển frame từ BGR sang RGB để phù hợp với PIL
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))

    cap.release()

    # Lưu GIF bằng PIL
    if frames:
        output_path = os.path.join(output_folder, os.path.splitext(os.path.basename(video_path))[0] + ".gif")
        frames[0].save(output_path, save_all=True, append_images=frames[1:], duration=int(1000 / fps), loop=0)

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    progress = tqdm(total=len(video_files), desc="Processing videos", unit="video")

    def process_video(video_file):
        video_path = os.path.join(input_folder, video_file)
        video_to_gif(video_path, output_folder)
        progress.update(1)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_video, video_files)

    progress.close()

if __name__ == "__main__":
    input_folder = "Thủ ngữ tiếng Anh"  # Đường dẫn tới thư mục chứa video
    output_folder = "output_gifs"  # Đường dẫn lưu file GIF

    if not os.path.exists(input_folder):
        print(f"Input folder '{input_folder}' does not exist.")
    else:
        process_folder(input_folder, output_folder)
        print(f"All videos have been processed and saved to '{output_folder}'.")


Processing videos: 100%|██████████| 5/5 [00:39<00:00,  7.89s/video]

All videos have been processed and saved to 'output_gifs'.
